In [1]:
import ipycytoscape as ic  # visualise
import pandas as pd
import netaddr
from stellargraph import StellarGraph
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans  # sanity check embeddings
from stellargraph.data import UnsupervisedSampler
from stellargraph.mapper import Attri2VecLinkGenerator
from stellargraph.layer import Attri2Vec, link_classification
from tensorflow import keras
from stellargraph.mapper import Attri2VecNodeGenerator  # for getting embeddings

### Make some data

Create some data in pandas for stellagraph ingestion. Using https://stellargraph.readthedocs.io/en/stable/demos/basics/loading-pandas.html

In [2]:
# cytoscape format
data = {'nodes': 
         [{'data': {'id': 'n0', 'name': 'Node 0', 'ip': '100.50.0.10', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n1', 'name': 'Node 1', 'ip': '10.1.10.201', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n2', 'name': 'Node 2', 'ip': '192.168.0.9', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n3', 'name': 'Node 3', 'ip': '192.168.0.56', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n4', 'name': 'Node 4', 'ip': '192.168.0.12', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n5', 'name': 'Node 5', 'ip': '100.50.1.30', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n6', 'name': 'Node 6', 'ip': '10.0.1.56', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n7', 'name': 'Node 7', 'ip': '10.0.1.2', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n8', 'name': 'Node 8', 'ip': '10.0.1.102', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n9', 'name': 'Node 9', 'ip': '10.0.1.100', 'mask': '255.255.255.0'}}],
        'edges': 
        [{'data': {'id': 'n0-n5', 'source': 'n0', 'target': 'n5'}},
         {'data': {'id': 'n0-n1', 'source': 'n0', 'target': 'n1'}},
         {'data': {'id': 'n0-n2', 'source': 'n0', 'target': 'n2'}},
         {'data': {'id': 'n0-n8', 'source': 'n0', 'target': 'n8'}},
         {'data': {'id': 'n0-n9', 'source': 'n0', 'target': 'n9'}},
         {'data': {'id': 'n1-n2', 'source': 'n1', 'target': 'n2'}},
         {'data': {'id': 'n8-n9', 'source': 'n8', 'target': 'n9'}},
         {'data': {'id': 'n5-n3', 'source': 'n5', 'target': 'n3'}},
         {'data': {'id': 'n5-n4', 'source': 'n5', 'target': 'n4'}},
         {'data': {'id': 'n5-n6', 'source': 'n5', 'target': 'n6'}},
         {'data': {'id': 'n5-n7', 'source': 'n5', 'target': 'n7'}},
         {'data': {'id': 'n3-n4', 'source': 'n3', 'target': 'n4'}},
         {'data': {'id': 'n6-n7', 'source': 'n6', 'target': 'n7'}}]
       }

In [3]:
edges_df = pd.DataFrame([x['data'] for x in data['edges']])
edges_df = edges_df.drop(['id'], axis=1)

In [4]:
nodes_df = pd.DataFrame([x['data'] for x in data['nodes']])
nodes_df = nodes_df.set_index(['id'])

StellarGraph only takes node features as a numerical type, so need to cast strings to numeric

In [5]:
networks = [netaddr.IPNetwork(x[0] + '/' + x[1]) for x in zip(nodes_df['ip'], nodes_df['mask'])]
nodes_df['ip_num'] = [int(x.ip.bits().replace('.',''), 2) for x in networks]
nodes_df['mask_num'] = [int(x.netmask.bits().replace('.',''), 2) for x in networks]
nodes_df = nodes_df.drop(['ip', 'mask', 'name'], axis=1)
nodes_df

,ip_num,mask_num
id,,
n0,1680998410,4294967040
n1,167840457,4294967040
n2,3232235529,4294967040
n3,3232235576,4294967040
n4,3232235532,4294967040
n5,1680998686,4294967040
n6,167772472,4294967040
n7,167772418,4294967040
n8,167772518,4294967040


In [6]:
nodes_edges_sg = StellarGraph(nodes_df, edges_df)
print(nodes_edges_sg.info())

StellarGraph: Undirected multigraph
 Nodes: 10, Edges: 13

 Node types:
  default: [10]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [13]
        Weights: all 1 (default)
        Features: none


### Attri2Vec

Here, we use attri2vec to convert the nodes to vector embeddings. Follows https://stellargraph.readthedocs.io/en/stable/demos/embeddings/attri2vec-embeddings.html


create a generator that will spit out node pairs that are sampled from the input graph

In [7]:
nodes = list(nodes_edges_sg.nodes())
number_of_walks = 4
length = 5
unsupervised_samples = UnsupervisedSampler(
    nodes_edges_sg, nodes=nodes, length=length, number_of_walks=number_of_walks
)
# help(unsupervised_samples)  # generates node pairs with a random binary label. Equal chance of 0 or 1 label. 
# help(GraphSAGELinkGenerator)

In [8]:
batch_size = 150
num_samples = [10, 5]  # sampling from 1-hop and 2-hop in graph
generator = Attri2VecLinkGenerator(nodes_edges_sg, batch_size, num_samples)
train_gen = generator.flow(unsupervised_samples)
# help(GraphSAGE)  # creates a two layer model

Create a Attri2vec model, which will have a single hidden layer

In [9]:
layer_sizes = [20]  # size of hidden layer will represent the dimensionality of embeddings
attri2vec = Attri2Vec(
    layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None
)
# Build the model and expose input and output sockets of graphsage, for node pair inputs:
x_inp, x_out = attri2vec.in_out_tensors()

Phrase the 'link prediction problem', which will be predicting the binary label on node pairs coming from unsupervised_samples. Note, the link prediction problem is just a means to an end of computing 2-hop kernel functions over nodes.

In [10]:
# Build the model and expose input and output sockets of graphsage, for node pair inputs:
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [11]:
# spec the model for training
model = keras.Model(inputs=x_inp, outputs=prediction)
model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

In [12]:
epochs = 10
history = model.fit(
    train_gen,
    epochs=epochs,
    verbose=1,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
)

Epoch 1/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6923 - binary_accuracy: 0.5156
Epoch 2/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6920 - binary_accuracy: 0.5250
Epoch 3/10
3/3 [==============================] - 0s 2ms/step - loss: 0.6895 - binary_accuracy: 0.5406
Epoch 4/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6908 - binary_accuracy: 0.5469
Epoch 5/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6923 - binary_accuracy: 0.5125
Epoch 6/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6891 - binary_accuracy: 0.5281
Epoch 7/10
3/3 [==============================] - 0s 2ms/step - loss: 0.6905 - binary_accuracy: 0.5500
Epoch 8/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6862 - binary_accuracy: 0.5625
Epoch 9/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6919 - binary_accuracy: 0.5406
Epoch 10/10
3/3 [==============================] - 0s 2ms/step - loss: 0.

Generate some node embeddings from the GraphSAGE layer stack

In [13]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)


In [14]:
node_ids = nodes_df.index.values.tolist()
node_gen = Attri2VecNodeGenerator(nodes_edges_sg, batch_size, num_samples).flow(node_ids)
node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

1/1 [==============================] - 0s 101ms/step


In [15]:
print(node_embeddings)

[[1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.]
 [1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.]
 [1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.]
 [1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.]
 [1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.]
 [1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0.]]


Use KMeans to look at embedding clusters. Output vectors all 1s and 0s. Clustering places heavy emphasis on node attributes.

In [16]:
X = node_embeddings
kmeans = KMeans(n_clusters=4).fit(X)
node_to_label = {x:y for x,y in zip(node_ids, kmeans.labels_)}
for label in set(kmeans.labels_):
    print([x for x, y in node_to_label.items() if y == label])

['n2', 'n3', 'n4']
['n1', 'n6', 'n7', 'n8', 'n9']
['n0', 'n5']


<ipython-input-16-356b48ea6ca0>:2: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=4).fit(X)


In [17]:
cytoscapeobj = ic.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(data)
cytoscapeobj.set_style([{
                            'selector': 'node',
                            'css': {
                                'content': 'data(id)',
                                'text-valign': 'center',
                                'color': 'black'
                            }
                        }])
display(cytoscapeobj)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'content': 'd…